<a href="https://colab.research.google.com/github/moaaz12-web/NER-using-Roberta/blob/main/NER_using_Roberta_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING DEPENDENCIES

In [ ]:
!pip install transformers datasets tokenizers seqeval evaluate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 873.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 14.7 MB/s eta 0:

In [ ]:
import datasets 
import numpy as np 
from transformers import AutoTokenizer 
from transformers import DataCollatorForTokenClassification 
from transformers import AutoModelForTokenClassification 

conll2003 = datasets.load_dataset("conll2003") 

  0%|          | 0/3 [00:00<?, ?it/s]

# INSTANTIATING ROBERTA TOKENIZER AND MODEL

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('Jean-Baptiste/roberta-large-ner-english')
model = AutoModelForTokenClassification.from_pretrained('Jean-Baptiste/roberta-large-ner-english', num_labels=9, ignore_mismatched_sizes=True)
model = model.to('cuda')


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at Jean-Baptiste/roberta-large-ner-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 1024]) in the checkpoint and torch.Size([9, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# DATA TOKENIZATION

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 


In [ ]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

# TRAINING ARGUMENTS

In [ ]:
from transformers import TrainingArguments, Trainer 
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=8, 
per_device_eval_batch_size=8, 
num_train_epochs=1, 
weight_decay=0.01, 
) 

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer) 
metric = datasets.load_metric("seqeval") 

<ipython-input-6-eac0b0fdfb3f>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


In [ ]:
example = conll2003['train'][0]
label_list = conll2003["train"].features["ner_tags"].feature.names 
labels = [label_list[i] for i in example["ner_tags"]] 

metric.compute(predictions=[labels], references=[labels]) 

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

# METRICS FUNCTION

In [ ]:
def compute_metrics(eval_preds): 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    # We remove all the values where the label is -100
    predictions = [ 
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

# TRAIN

In [ ]:
CUDA_LAUNCH_BLOCKING=1
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<512>"
%env PYTORCH_CUDA_ALLOC_CONF=512


trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 
trainer.train() 

env: PYTORCH_CUDA_ALLOC_CONF=512


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.033300,0.052154,0.959831,0.965905,0.962858,0.990442


TrainOutput(global_step=1756, training_loss=0.043222241629772146, metrics={'train_runtime': 696.0775, 'train_samples_per_second': 20.172, 'train_steps_per_second': 2.523, 'total_flos': 1063351209445104.0, 'train_loss': 0.043222241629772146, 'epoch': 1.0})

# VALIDATE

In [ ]:
res = trainer.evaluate()

In [ ]:
res

{'eval_loss': 0.0521538183093071,
 'eval_precision': 0.9598310652275927,
 'eval_recall': 0.9659047978843974,
 'eval_f1': 0.9628583533399238,
 'eval_accuracy': 0.9904421466048909,
 'eval_runtime': 34.6275,
 'eval_samples_per_second': 93.856,
 'eval_steps_per_second': 11.754,
 'epoch': 1.0}

# SAVE VALIDATION RESULTS TO A FOLDER

In [ ]:
import os
import csv
import time

# Create the Validation results folder if it doesn't exist
if not os.path.exists('Validation results'):
    os.makedirs('Validation results')

# Define the header and data for the CSV file
header = ['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1', 'eval_accuracy', 'eval_runtime', 'eval_samples_per_second', 'eval_steps_per_second', 'epoch']
data = [str(value) for key, value in res.items()]

# Generate a unique filename for the CSV file based on the current timestamp
timestamp = int(time.time())
filename = time.strftime("results_%Y-%m-%d_%H-%M-%S.csv", time.localtime(timestamp))

# Save the data to the CSV file
with open(f'Validation results/{filename}', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerow(data)


# SAVING MODEL AND TOKENIZER

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [ ]:
model.save_pretrained("ner_model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

# INFERENCE

In [ ]:
import json
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [ ]:
from transformers import pipeline
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Bill Gates is the Founder of Microsoft"

ner_results = nlp(example)

print(ner_results)

[{'entity': 'B-PER', 'score': 0.99957126, 'index': 1, 'word': 'ĠBill', 'start': 0, 'end': 4}, {'entity': 'I-PER', 'score': 0.99954176, 'index': 2, 'word': 'ĠGates', 'start': 5, 'end': 10}, {'entity': 'B-ORG', 'score': 0.99888223, 'index': 7, 'word': 'ĠMicrosoft', 'start': 29, 'end': 38}]
